In [1]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

"""
Kamu adalah seorang konselor virtual yang ramah, empatik, dan sangat peduli terhadap kesehatan mental. 
Berikan jawaban dengan bahasa yang hangat, tidak menghakimi, dan mendukung.

Informasi tambahan: 
{context}

Pertanyaan dari pengguna:
{question}

Berikan jawaban yang membantu, berempati, dan menguatkan.
"""

Kamu adalah konselor profesional yang penuh empati. Berdasarkan konteks berikut, berikan respons singkat, manusiawi, dan berfokus pada inti masalah pengguna.

Gunakan bahasa yang sederhana, suportif, dan tidak menggurui.

Setelah memberikan sedikit pemahaman atau dukungan, akhiri dengan kalimat yang membuka ruang untuk pengguna berbagi lebih banyak atau memperdalam pembicaraan.

In [7]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
"""
Kamu adalah konselor profesional yang penuh empati. Berdasarkan konteks berikut, berikan respons singkat, manusiawi, dan berfokus pada inti masalah pengguna.
Gunakan bahasa yang sederhana, suportif, dan tidak menggurui.

Informasi tambahan: 
{context}

Pertanyaan dari pengguna:
{question}

Setelah memberikan sedikit pemahaman atau dukungan, akhiri dengan kalimat yang membuka ruang untuk pengguna berbagi lebih banyak atau memperdalam pembicaraan.
"""
)


In [3]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
import torch
from langchain.schema import Document

# Load SBERT model dan pastikan menggunakan GPU jika tersedia
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sbert_model = model = SentenceTransformer('naufalihsan/indonesian-sbert-large')
sbert_model = sbert_model.to(device)  # Pindahkan model ke GPU (jika ada)

# Custom embeddings class for SBERT
class SBERTEmbeddings(Embeddings):
    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        # Menggunakan model SBERT untuk menghasilkan embeddings
        embeddings = sbert_model.encode(texts, convert_to_tensor=True, show_progress_bar=True)
        embeddings = embeddings.to(device)  # Pindahkan embeddings ke GPU (jika ada)
        return embeddings.cpu().numpy().tolist()  # Pindahkan kembali ke CPU untuk konversi

    def embed_query(self, query: str) -> list[float]:
        # Menghasilkan embedding untuk query
        embedding = sbert_model.encode(query, convert_to_tensor=True)
        embedding = embedding.to(device)  # Pindahkan embedding ke GPU (jika ada)
        return embedding.cpu().numpy().tolist()  # Pindahkan kembali ke CPU untuk konversi

# Inisialisasi embeddings SBERT dan FAISS vector store
sbert_embeddings = SBERTEmbeddings()

d:\CODING\PYTHON\MACHINE_LEARNING\Tenangin\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\CODING\PYTHON\MACHINE_LEARNING\Tenangin\.venv\Lib\site-packages\transformers\utils\hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [8]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
vector_store = FAISS.load_local("tudocs_faiss",sbert_embeddings,allow_dangerous_deserialization=True)


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [9]:
response = graph.invoke({"question": "aku ngerasa sedih banget hari ini huhu"})
print(f"Anda    : {response['question']}")
print(f"Gemini  : {response['answer']}")
print(f"Context : {response['context'][0].page_content}")

Anda    : aku ngerasa sedih banget hari ini huhu
Gemini  : Aku turut merasakan kesedihanmu. Sedih itu emosi yang wajar kok, dan nggak apa-apa untuk merasakannya. Kadang, hidup memang terasa berat ya?

Apakah ada sesuatu spesifik yang membuatmu merasa sedih hari ini? Ceritakan saja jika kamu merasa nyaman.
Context : lainnya (Nierenberg, 2001). Gangguan mood adalah disfungsi neurobiologi yang menyebabkan perubahan respons emosional. Gangguan mood mengganggu kehidupan seseorang. Orang-orang dipenuhi dengan kesedihan, kegembiraan atau kegembiraan yang berkepanjangan dan intens, disertai dengan keraguan diri, rasa bersalah dan kemarahan, yang mengubah aktivitas hidup mereka, terutama aktivitas yang berkaitan dengan harga diri, pekerjaan, dan hubungan (Videbeck, 2011), dan ketika kondisi ini berlangsung lama dan terus menerus, orang bisa mengalami keputusasaan dan bunuh diri. Kematian akibat bunuh diri dapat dicegah dan dapat mengurangi kejadian bunuh diri. Memahami, mendeteksi, dan mengobat